In [144]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline
import requests
import lxml.html as lh
from urllib.request import urlopen

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: - ^C
failed

CondaError: KeyboardInterrupt

Libraries imported.


# Question 1: Transform the data in the table on the Wikipedia page into the above pandas dataframe

## First, read the table from the URL using pandas. Define the table as df. 

In [146]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
all_tables=pd.read_html(url)
df=all_tables[0]
df.shape

(180, 3)

In [147]:
df.head(5)

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


### Exclude cells with a borough that is "Not assigned". 

In [148]:
df=df[df.Borough!="Not assigned"]

### Next, verify that there is only one line for each postal code. 

In [149]:
len(df["Postal code"].unique())==len(df["Postal code"])

True

### Replace the character "/" with "," for postal codes with multiple neighborhoods. 

In [150]:
df["Neighborhood"]=df["Neighborhood"].str.replace(" /", ",").tolist()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


### Assign the borough name to neighborhood name for rows with blank neighborhood and non-blank borough (although there was no such row in the dataset at this stage). 

In [151]:
df[(df["Neighborhood"]=="") & (df["Borough"]!="Not assigned")].Neighborhood=df[(df["Neighborhood"]=="") & (df["Borough"]!="Not assigned")].Borough

### Reset the row indices and print out a sample of the dataset to check the structure

In [152]:
df.reset_index(drop=True, inplace=True)
df.head(10)

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


### Print the dimensions of the dataset. 

In [153]:
df.shape

(103, 3)

# Question 2: Create the dataframe with latitude and longitude columns. 

In [154]:
## As the geocoder package caused problems, I used the provided csv file for coordinates of each postal code. 
!wget -O coordinates.csv https://cocl.us/Geospatial_data

--2020-04-29 13:38:38--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.194, 158.85.108.83, 158.85.108.86
Connecting to cocl.us (cocl.us)|169.48.113.194|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-04-29 13:38:39--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-04-29 13:38:39--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr

### Read the cvs file for coordinates of each postal code. 

In [155]:
coordinates = pd.read_csv('coordinates.csv')
coordinates.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


### Merge the df and coordinates dataframes based on the postal code and drop the duplicate postal code column. 

In [156]:
df_coordinates= pd.merge(left=df, right=coordinates, left_on='Postal code', right_on='Postal Code')
df_coordinates.drop('Postal Code', axis=1, inplace=True)
df_coordinates.head(10)

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# Question 3: Explore and cluster the neighborhoods in Toronto. 

### First, I'll use geopy library to create a map of Toronto with neighborhoods superimposed on top. 

### Start by obtaining the coordinates of Toronto. 

In [157]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Now create the Toronto map with neighborhoods in the df_coorfinates data frame. 

In [158]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_coordinates['Latitude'], df_coordinates['Longitude'], df_coordinates['Borough'], df_coordinates['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Next, I'll use the Foursquare API to explore the neighborhoods and segment them. 

### First, define Foursquare credentials. 

In [159]:
CLIENT_ID = 'SKEQ5AEZ3B13FFWQDJYY11GPGUN1WJE5MMCOG3HLTL0F5B4K' # your Foursquare ID
CLIENT_SECRET = 'YIQBBQ5IFRC0IXLRQLB1V1ZCHABFDDJ3HY1QGJI4LUPYWMQ3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SKEQ5AEZ3B13FFWQDJYY11GPGUN1WJE5MMCOG3HLTL0F5B4K
CLIENT_SECRET:YIQBBQ5IFRC0IXLRQLB1V1ZCHABFDDJ3HY1QGJI4LUPYWMQ3


### Next I will explore the neighborhoods in Toronto. I will only work with boroughs that include the word "Toronto". I define this subset of the data frame as df_subset, reset its index and print the first 10 lines for illustration. 

In [160]:
df_subset=df_coordinates[df_coordinates.Borough.str.contains("Toronto")]
df_subset.reset_index(drop=True, inplace=True)
df_subset.head(10)

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


### Let's create a function to get nearby venues.

In [161]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
           'SKEQ5AEZ3B13FFWQDJYY11GPGUN1WJE5MMCOG3HLTL0F5B4K',
'YIQBBQ5IFRC0IXLRQLB1V1ZCHABFDDJ3HY1QGJI4LUPYWMQ3',  
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [84]:
toronto_venues = getNearbyVenues(names=df_subset['Neighborhood'],
                                   latitudes=df_subset['Latitude'],
                                   longitudes=df_subset['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town,

### Let's check the size of the resulting dataframe. 

In [162]:
print(toronto_venues.shape)
toronto_venues.head(10)

(1636, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
5,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
6,"Regent Park, Harbourfront",43.65426,-79.360636,Corktown Common,43.655618,-79.356211,Park
7,"Regent Park, Harbourfront",43.65426,-79.360636,The Extension Room,43.653313,-79.359725,Gym / Fitness Center
8,"Regent Park, Harbourfront",43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site
9,"Regent Park, Harbourfront",43.65426,-79.360636,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot


### Let's check how many venues were returned for each neighborhood

In [163]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
Business reply mail Processing CentrE,16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",18,18,18,18,18,18
Central Bay Street,64,64,64,64,64,64
Christie,18,18,18,18,18,18
Church and Wellesley,76,76,76,76,76,76
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,37,37,37,37,37,37


### Let's find out how many unique categories can be curated from all the returned venues

In [164]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 234 uniques categories.


## Next I will analyze each neighborhood based on the frequency of the type of venues in the area. 

In [165]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

#find the location of Neighborhood column
col_locn=toronto_onehot.columns.get_loc("Neighborhood")

# move neighborhood column at index "col_locn" to the first column
fixed_columns = [toronto_onehot.columns[col_locn]] + list(toronto_onehot.columns[toronto_onehot.columns!="Neighborhood"])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head(5)


(1636, 234)


,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [166]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped.head(5)

(39, 234)


,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.017241,0.0,0.0,0.0,0.0,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
2,Business reply mail Processing CentrE,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.015625,0.0,0.0,0.0,0.0,0.0,0.015625


### Let's view the top 10 most common venue types in each neighborhood in dataframe df_subset. 

### First, let's write a function to sort the venues in descending order.

In [167]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [219]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Seafood Restaurant,Bakery,Restaurant,Italian Restaurant,Beer Bar,Farmers Market
1,"Brockton, Parkdale Village, Exhibition Place",Café,Nightclub,Breakfast Spot,Coffee Shop,Stadium,Bar,Intersection,Bakery,Italian Restaurant,Restaurant
2,Business reply mail Processing CentrE,Light Rail Station,Smoke Shop,Pizza Place,Fast Food Restaurant,Farmers Market,Auto Workshop,Burrito Place,Spa,Restaurant,Comic Shop
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Airport,Bar,Harbor / Marina,Plane,Rental Car Location,Sculpture Garden,Boutique
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Ice Cream Shop,Bar,Middle Eastern Restaurant,Fried Chicken Joint,Salad Place,Thai Restaurant
5,Christie,Grocery Store,Café,Park,Baby Store,Nightclub,Restaurant,Italian Restaurant,Athletics & Sports,Diner,Candy Store
6,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Yoga Studio,Gay Bar,Pub,Hotel,Men's Store,Mediterranean Restaurant
7,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Restaurant,Hotel,Gym,American Restaurant,Deli / Bodega,Seafood Restaurant,Italian Restaurant,Japanese Restaurant
8,Davisville,Sandwich Place,Pizza Place,Dessert Shop,Coffee Shop,Gym,Sushi Restaurant,Café,Italian Restaurant,Toy / Game Store,Thai Restaurant
9,Davisville North,Park,Convenience Store,Department Store,Hotel,Food & Drink Shop,Sandwich Place,Gym,Breakfast Spot,Dog Run,Distribution Center


## Finally, I will cluster the neighborhoods based on the type of top 10 venues in the area. 

In [220]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:30] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 0, 2, 1, 1], dtype=int32)

In [221]:
 #add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_subset[["Neighborhood", "Latitude", "Longitude"]]

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Restaurant,Café,Theater,Mexican Restaurant,Shoe Store
1,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Creperie,Burger Joint,Burrito Place,Restaurant,Café,General Entertainment
2,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Japanese Restaurant,Middle Eastern Restaurant,Cosmetics Shop,Restaurant,Lingerie Store,Ramen Restaurant
3,St. James Town,43.651494,-79.375418,1,Café,Coffee Shop,Italian Restaurant,Cocktail Bar,Gastropub,American Restaurant,Theater,Gym,Department Store,Hotel
4,The Beaches,43.676357,-79.293031,1,Trail,Health Food Store,Pub,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Yoga Studio,Cupcake Shop


## Finally, let's visualize the resulting clusters.

In [222]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## I've experimented with different number of clusters. But the observation was the same: The vast majority of neighborhoods were in the same cluster. Except for this majority cluster, remaining ones has one or two neighborhoods in it. To understand what's the characteristic of each cluster, I'll next analyze each cluster separately. I've also set the number of clusters to 3 as this resulted in the most meaningful separation based on the individual cluster analysis below.  

### Cluster 1

In [223]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Lawrence Park,Park,Bus Line,Swim School,Event Space,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
21,Forest Hill North & West,Park,Jewelry Store,Trail,Bus Line,Sushi Restaurant,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Donut Shop,Doner Restaurant
29,"Moore Park, Summerhill East",Park,Restaurant,Playground,Tennis Court,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
33,Rosedale,Park,Playground,Trail,Cuban Restaurant,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


### Cluster 2

In [224]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Restaurant,Café,Theater,Mexican Restaurant,Shoe Store
1,"Queen's Park, Ontario Provincial Government",Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Creperie,Burger Joint,Burrito Place,Restaurant,Café,General Entertainment
2,"Garden District, Ryerson",Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Japanese Restaurant,Middle Eastern Restaurant,Cosmetics Shop,Restaurant,Lingerie Store,Ramen Restaurant
3,St. James Town,Café,Coffee Shop,Italian Restaurant,Cocktail Bar,Gastropub,American Restaurant,Theater,Gym,Department Store,Hotel
4,The Beaches,Trail,Health Food Store,Pub,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Yoga Studio,Cupcake Shop
5,Berczy Park,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Seafood Restaurant,Bakery,Restaurant,Italian Restaurant,Beer Bar,Farmers Market
6,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Ice Cream Shop,Bar,Middle Eastern Restaurant,Fried Chicken Joint,Salad Place,Thai Restaurant
7,Christie,Grocery Store,Café,Park,Baby Store,Nightclub,Restaurant,Italian Restaurant,Athletics & Sports,Diner,Candy Store
8,"Richmond, Adelaide, King",Coffee Shop,Café,Restaurant,Gym,Clothing Store,Deli / Bodega,American Restaurant,Hotel,Thai Restaurant,Bookstore
9,"Dufferin, Dovercourt Village",Bakery,Pharmacy,Pool,Art Gallery,Liquor Store,Bus Stop,Café,Bar,Bank,Supermarket


### Cluster 3

In [225]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,Garden,Yoga Studio,Cupcake Shop,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


## Main observations from analyzing the top 10 venues in each neighborhood that has "Toronto" in the borough name: 
### 1.  Cluster 1 consists of 4 neighborhoods that are located close to a park and and have outdoor activity spaces such as trails. 
### 2. Cluster 2 includes the majority of neighborhoods in the dataset. This cluster represents more central areas that have coffee shops, hotels and restaurants of several cousines in the vicinity. 
### 3. Cluster 3 has a single neighborhood that most commonly has garden, yoga studio and cupcake shop. 